In [1]:
"""
Parcial 3

Computacion estadistica
UNAL 2023-1

Willian Andrés Cárdenas Sandoval
"""

'\nParcial 3\n\nComputacion estadistica\nUNAL 2023-1\n\nWillian Andrés Cárdenas Sandoval\n'

In [2]:
import os 
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mismacrospython import onewayfreq, twowayfreq
!pip install duckdb
!pip install openpyxl 
import duckdb
con = duckdb.connect(database=':memory:')

pd.options.display.float_format = '{:,.2f}'.format


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
games = pd.read_csv('games.csv')
appea = pd.read_csv('appearances.csv',quotechar="'", header = 0,names=['gameID', 'playerID', 'goals', 'ownGoals', 'shots',
       'xGoals', 'xGoalsChain', 'xGoalsBuildup', 'assists',
       'keyPasses', '""xAssists', 'position', 'positionOrder',
       'yellowCard', 'redCard', 'time', 'substituteIn',
       'substituteOut', 'leagueID'])
appea['gameID'] = appea.gameID.str.replace('"',"")
appea['playerID'] = appea.playerID.str.replace('"',"")
appea['leagueID'] = appea.playerID.str.replace('"',"")
uefa_l = pd.read_excel('UEFA5.xlsx',sheet_name="leagues",header=0, usecols="A:B")
uefa_t = pd.read_excel('UEFA5.xlsx',sheet_name="teams",header=0, usecols="A:B")
uefa_p = pd.read_excel('UEFA5.xlsx',sheet_name="players",header=0, usecols="A:B")

In [4]:
appear = con.execute("""select   appea.*, uefa_l.leagueNAME
                     from     appea left join uefa_l on (appea.leagueID = uefa_l.leagueID)
                     """).df()
appear = con.execute("""select   appear.*, uefa_p.playerNAME
                     from     appear left join uefa_p on (appear.playerID = uefa_p.playerID)
                     """).df()
games = con.execute("""select   games.*, uefa_l.leagueNAME
                     from     games left join uefa_l on (games.leagueID = uefa_l.leagueID)
                     """).df()

In [5]:
#Punto 1
e1 = con.execute("""select   games.*
                     from games
                     where season == 2016 and leagueID == 1
                     """).df()
e1a = con.execute("""select  distinct awayTeamID as teamID
                     from e1 
                     """).df()
e1a = con.execute("""select   e1a.*, uefa_t.teamNAME
                     from     e1a left join uefa_t on (e1a.teamID = uefa_t.teamID)
                     """).df()

team_h = con.execute("""select   e1a.*, case when e1.homeGoals > e1.awayGoals  then 'w'
                                             when e1.homeGoals < e1.awayGoals then 'l'
                                             else 'd'
                                        end as res,
                                        e1.homeGoals - e1.awayGoals as dg
                        from e1a  left join e1 on (e1a.teamID = e1.hometeamID)
                     """).df()

team_hw = con.execute("""select   team_h.*
                        from team_h
                        where res in ('w')
                     """).df()
team_hw = con.execute("""select teamID,teamNAME,count(*) as vic, sum(dg) as dg
                        from team_hw
                        group by teamID,teamNAME
                     """).df()

team_hd = con.execute("""select   team_h.*
                        from team_h
                        where res in ('d')
                     """).df()
team_hd = con.execute("""select teamID,teamNAME,count(*) as dra,sum(dg) as dg
                        from team_hd
                        group by teamID,teamNAME
                     """).df()

team_hl = con.execute("""select   team_h.*
                        from team_h
                        where res in ('l')
                     """).df()
team_hl = con.execute("""select teamID,teamNAME,count(*) as los,sum(dg) as dg
                        from team_hl
                        group by teamID,teamNAME
                     """).df()
#####
team_a = con.execute("""select   e1a.*, case when e1.homeGoals < e1.awayGoals  then 'w'
                                             when e1.homeGoals > e1.awayGoals then 'l'
                                             else 'd'
                                        end as res,
                                        e1.awayGoals - e1.homeGoals  as dg
                        from e1a  left join e1 on (e1a.teamID = e1.hometeamID)
                     """).df()

team_aw = con.execute("""select   team_a.*
                        from team_a
                        where res in ('w')
                     """).df()
team_aw = con.execute("""select teamID,teamNAME,count(*) as vic, sum(dg) as dg
                        from team_aw
                        group by teamID,teamNAME
                     """).df()

team_ad = con.execute("""select   team_a.*
                        from team_a
                        where res in ('d')
                     """).df()
team_ad = con.execute("""select teamID,teamNAME,count(*) as dra, sum(dg) as dg
                        from team_ad
                        group by teamID,teamNAME
                     """).df()

team_al = con.execute("""select   team_a.*
                        from team_a
                        where res in ('d')
                     """).df()
team_al = con.execute("""select teamID,teamNAME,count(*) as los, sum(dg) as dg
                        from team_al
                        group by teamID,teamNAME
                     """).df()
    
e1a["win"] = team_hw["vic"] + team_aw["vic"]
e1a["draw"] = team_hd["dra"] + team_ad["dra"]
e1a["puntos"] = 3* e1a["win"] + e1a["draw"]
e1a["dif_gol"] = team_hw["dg"] + team_hl["dg"] + team_aw["dg"] + team_al["dg"] 

e1a = con.execute("""select   e1a.*
                     from e1a
                     order by puntos desc
                     """).df()
e1a

,teamID,teamNAME,win,draw,puntos,dif_gol
0,93,Middlesbrough,NaN,NaN,NaN,NaN
1,75,Leicester,23.00,12.00,81.00,40.00
2,78,Crystal Palace,25.00,4.00,79.00,38.00
3,82,Tottenham,24.00,6.00,78.00,30.00
4,83,Arsenal,21.00,12.00,75.00,25.00
5,85,Stoke,21.00,8.00,71.00,29.00
6,72,Everton,19.00,8.00,65.00,12.00
7,74,Southampton,17.00,12.00,63.00,22.00
8,81,West Ham,18.00,6.00,60.00,20.00
9,87,Liverpool,17.00,8.00,59.00,18.00


In [6]:
#Punto 2
e1 = con.execute("""select   games.*
                     from games
                     where season == 2017 and leagueID == 2
                     """).df()
e1a = con.execute("""select  distinct awayTeamID as teamID
                     from e1 
                     """).df()
e1a = con.execute("""select   e1a.*, uefa_t.teamNAME
                     from     e1a left join uefa_t on (e1a.teamID = uefa_t.teamID)
                     """).df()

team_h = con.execute("""select   e1a.*, case when e1.homeGoals > e1.awayGoals  then 'w'
                                             when e1.homeGoals < e1.awayGoals then 'l'
                                             else 'd'
                                        end as res,
                                        e1.homeGoals - e1.awayGoals as dg
                        from e1a  left join e1 on (e1a.teamID = e1.hometeamID)
                     """).df()

team_hw = con.execute("""select   team_h.*
                        from team_h
                        where res in ('w')
                     """).df()
team_hw = con.execute("""select teamID,teamNAME,count(*) as vic, sum(dg) as dg
                        from team_hw
                        group by teamID,teamNAME
                     """).df()

team_hd = con.execute("""select   team_h.*
                        from team_h
                        where res in ('d')
                     """).df()
team_hd = con.execute("""select teamID,teamNAME,count(*) as dra,sum(dg) as dg
                        from team_hd
                        group by teamID,teamNAME
                     """).df()

team_hl = con.execute("""select   team_h.*
                        from team_h
                        where res in ('l')
                     """).df()
team_hl = con.execute("""select teamID,teamNAME,count(*) as los,sum(dg) as dg
                        from team_hl
                        group by teamID,teamNAME
                     """).df()
#####
team_a = con.execute("""select   e1a.*, case when e1.homeGoals < e1.awayGoals  then 'w'
                                             when e1.homeGoals > e1.awayGoals then 'l'
                                             else 'd'
                                        end as res,
                                        e1.awayGoals - e1.homeGoals  as dg
                        from e1a  left join e1 on (e1a.teamID = e1.hometeamID)
                     """).df()

team_aw = con.execute("""select   team_a.*
                        from team_a
                        where res in ('w')
                     """).df()
team_aw = con.execute("""select teamID,teamNAME,count(*) as vic, sum(dg) as dg
                        from team_aw
                        group by teamID,teamNAME
                     """).df()

team_ad = con.execute("""select   team_a.*
                        from team_a
                        where res in ('d')
                     """).df()
team_ad = con.execute("""select teamID,teamNAME,count(*) as dra, sum(dg) as dg
                        from team_ad
                        group by teamID,teamNAME
                     """).df()

team_al = con.execute("""select   team_a.*
                        from team_a
                        where res in ('d')
                     """).df()
team_al = con.execute("""select teamID,teamNAME,count(*) as los, sum(dg) as dg
                        from team_al
                        group by teamID,teamNAME
                     """).df()
    
e1a["win"] = team_hw["vic"] + team_aw["vic"]
e1a["draw"] = team_hd["dra"] + team_ad["dra"]
e1a["puntos"] = 3* e1a["win"] + e1a["draw"]
e1a["dif_gol"] = team_hw["dg"] + team_hl["dg"] + team_aw["dg"] + team_al["dg"] 

e1a = con.execute("""select   e1a.*
                     from e1a
                     order by puntos desc
                     """).df()
e1a

,teamID,teamNAME,win,draw,puntos,dif_gol
0,94,Verona,29,8,95,39.00
1,102,Sampdoria,26,6,84,28.00
2,96,Lazio,23,12,81,25.00
3,107,Atalanta,18,12,66,32.00
4,110,Fiorentina,18,10,64,15.00
5,101,Genoa,16,12,60,16.00
6,99,Udinese,15,12,57,7.00
7,95,Roma,15,10,55,25.00
8,111,AC Milan,16,6,54,21.00
9,109,Chievo,12,16,52,13.00


In [7]:
#Punto 3
e1 = con.execute("""select   games.*
                     from games
                     where season == 2018 and leagueID == 3
                     """).df()
e1a = con.execute("""select  distinct awayTeamID as teamID
                     from e1 
                     """).df()
e1a = con.execute("""select   e1a.*, uefa_t.teamNAME
                     from     e1a left join uefa_t on (e1a.teamID = uefa_t.teamID)
                     """).df()

team_h = con.execute("""select   e1a.*, case when e1.homeGoals > e1.awayGoals  then 'w'
                                             when e1.homeGoals < e1.awayGoals then 'l'
                                             else 'd'
                                        end as res,
                                        e1.homeGoals - e1.awayGoals as dg
                        from e1a  left join e1 on (e1a.teamID = e1.hometeamID)
                     """).df()

team_hw = con.execute("""select   team_h.*
                        from team_h
                        where res in ('w')
                     """).df()
team_hw = con.execute("""select teamID,teamNAME,count(*) as vic, sum(dg) as dg
                        from team_hw
                        group by teamID,teamNAME
                     """).df()

team_hd = con.execute("""select   team_h.*
                        from team_h
                        where res in ('d')
                     """).df()
team_hd = con.execute("""select teamID,teamNAME,count(*) as dra,sum(dg) as dg
                        from team_hd
                        group by teamID,teamNAME
                     """).df()

team_hl = con.execute("""select   team_h.*
                        from team_h
                        where res in ('l')
                     """).df()
team_hl = con.execute("""select teamID,teamNAME,count(*) as los,sum(dg) as dg
                        from team_hl
                        group by teamID,teamNAME
                     """).df()
#####
team_a = con.execute("""select   e1a.*, case when e1.homeGoals < e1.awayGoals  then 'w'
                                             when e1.homeGoals > e1.awayGoals then 'l'
                                             else 'd'
                                        end as res,
                                        e1.awayGoals - e1.homeGoals  as dg
                        from e1a  left join e1 on (e1a.teamID = e1.hometeamID)
                     """).df()

team_aw = con.execute("""select   team_a.*
                        from team_a
                        where res in ('w')
                     """).df()
team_aw = con.execute("""select teamID,teamNAME,count(*) as vic, sum(dg) as dg
                        from team_aw
                        group by teamID,teamNAME
                     """).df()

team_ad = con.execute("""select   team_a.*
                        from team_a
                        where res in ('d')
                     """).df()
team_ad = con.execute("""select teamID,teamNAME,count(*) as dra, sum(dg) as dg
                        from team_ad
                        group by teamID,teamNAME
                     """).df()

team_al = con.execute("""select   team_a.*
                        from team_a
                        where res in ('d')
                     """).df()
team_al = con.execute("""select teamID,teamNAME,count(*) as los, sum(dg) as dg
                        from team_al
                        group by teamID,teamNAME
                     """).df()
    
e1a["win"] = team_hw["vic"] + team_aw["vic"]
e1a["draw"] = team_hd["dra"] + team_ad["dra"]
e1a["puntos"] = 3* e1a["win"] + e1a["draw"]
e1a["dif_gol"] = team_hw["dg"] + team_hl["dg"] + team_aw["dg"] + team_al["dg"] 

e1a = con.execute("""select   e1a.*
                     from e1a
                     order by puntos desc
                     """).df()
e1a

,teamID,teamNAME,win,draw,puntos,dif_gol
0,234,Nuernberg,6,NaN,NaN,NaN
1,123,Werder Bremen,24,10.00,82.00,29.00
2,136,Bayern Munich,21,12.00,75.00,38.00
3,128,Hannover 96,20,10.00,70.00,17.00
4,124,Schalke 04,15,14.00,59.00,16.00
5,122,Hertha Berlin,15,12.00,57.00,16.00
6,131,Wolfsburg,15,10.00,55.00,16.00
7,121,Augsburg,15,8.00,53.00,22.00
8,125,Mainz 05,14,8.00,50.00,26.00
9,120,Hoffenheim,11,14.00,47.00,9.00


In [8]:
#Punto 4
e1 = con.execute("""select   games.*
                     from games
                     where season == 2019 and leagueID == 4
                     """).df()
e1a = con.execute("""select  distinct awayTeamID as teamID
                     from e1 
                     """).df()
e1a = con.execute("""select   e1a.*, uefa_t.teamNAME
                     from     e1a left join uefa_t on (e1a.teamID = uefa_t.teamID)
                     """).df()

team_h = con.execute("""select   e1a.*, case when e1.homeGoals > e1.awayGoals  then 'w'
                                             when e1.homeGoals < e1.awayGoals then 'l'
                                             else 'd'
                                        end as res,
                                        e1.homeGoals - e1.awayGoals as dg
                        from e1a  left join e1 on (e1a.teamID = e1.hometeamID)
                     """).df()

team_hw = con.execute("""select   team_h.*
                        from team_h
                        where res in ('w')
                     """).df()
team_hw = con.execute("""select teamID,teamNAME,count(*) as vic, sum(dg) as dg
                        from team_hw
                        group by teamID,teamNAME
                     """).df()

team_hd = con.execute("""select   team_h.*
                        from team_h
                        where res in ('d')
                     """).df()
team_hd = con.execute("""select teamID,teamNAME,count(*) as dra,sum(dg) as dg
                        from team_hd
                        group by teamID,teamNAME
                     """).df()

team_hl = con.execute("""select   team_h.*
                        from team_h
                        where res in ('l')
                     """).df()
team_hl = con.execute("""select teamID,teamNAME,count(*) as los,sum(dg) as dg
                        from team_hl
                        group by teamID,teamNAME
                     """).df()
#####
team_a = con.execute("""select   e1a.*, case when e1.homeGoals < e1.awayGoals  then 'w'
                                             when e1.homeGoals > e1.awayGoals then 'l'
                                             else 'd'
                                        end as res,
                                        e1.awayGoals - e1.homeGoals  as dg
                        from e1a  left join e1 on (e1a.teamID = e1.hometeamID)
                     """).df()

team_aw = con.execute("""select   team_a.*
                        from team_a
                        where res in ('w')
                     """).df()
team_aw = con.execute("""select teamID,teamNAME,count(*) as vic, sum(dg) as dg
                        from team_aw
                        group by teamID,teamNAME
                     """).df()

team_ad = con.execute("""select   team_a.*
                        from team_a
                        where res in ('d')
                     """).df()
team_ad = con.execute("""select teamID,teamNAME,count(*) as dra, sum(dg) as dg
                        from team_ad
                        group by teamID,teamNAME
                     """).df()

team_al = con.execute("""select   team_a.*
                        from team_a
                        where res in ('d')
                     """).df()
team_al = con.execute("""select teamID,teamNAME,count(*) as los, sum(dg) as dg
                        from team_al
                        group by teamID,teamNAME
                     """).df()
    
e1a["win"] = team_hw["vic"] + team_aw["vic"]
e1a["draw"] = team_hd["dra"] + team_ad["dra"]
e1a["puntos"] = 3* e1a["win"] + e1a["draw"]
e1a["dif_gol"] = team_hw["dg"] + team_hl["dg"] + team_aw["dg"] + team_al["dg"] 

e1a = con.execute("""select   e1a.*
                     from e1a
                     order by puntos desc
                     """).df()
e1a

,teamID,teamNAME,win,draw,puntos,dif_gol
0,142,Getafe,NaN,8,NaN,NaN
1,147,Athletic Club,24.00,12,84.00,37.00
2,141,Espanyol,23.00,14,83.00,18.00
3,138,Real Madrid,19.00,8,65.00,15.00
4,152,Celta Vigo,14.00,20,62.00,19.00
5,154,Villarreal,16.00,12,60.00,16.00
6,148,Barcelona,15.00,14,59.00,14.00
7,140,Real Sociedad,16.00,10,58.00,12.00
8,146,Valencia,15.00,12,57.00,14.00
9,151,Levante,17.00,6,57.00,29.00


In [9]:
#Punto 5
e1 = con.execute("""select   games.*
                     from games
                     where season == 2020 and leagueID == 5
                     """).df()
e1a = con.execute("""select  distinct awayTeamID as teamID
                     from e1 
                     """).df()
e1a = con.execute("""select   e1a.*, uefa_t.teamNAME
                     from     e1a left join uefa_t on (e1a.teamID = uefa_t.teamID)
                     """).df()

team_h = con.execute("""select   e1a.*, case when e1.homeGoals > e1.awayGoals  then 'w'
                                             when e1.homeGoals < e1.awayGoals then 'l'
                                             else 'd'
                                        end as res,
                                        e1.homeGoals - e1.awayGoals as dg
                        from e1a  left join e1 on (e1a.teamID = e1.hometeamID)
                     """).df()

team_hw = con.execute("""select   team_h.*
                        from team_h
                        where res in ('w')
                     """).df()
team_hw = con.execute("""select teamID,teamNAME,count(*) as vic, sum(dg) as dg
                        from team_hw
                        group by teamID,teamNAME
                     """).df()

team_hd = con.execute("""select   team_h.*
                        from team_h
                        where res in ('d')
                     """).df()
team_hd = con.execute("""select teamID,teamNAME,count(*) as dra,sum(dg) as dg
                        from team_hd
                        group by teamID,teamNAME
                     """).df()

team_hl = con.execute("""select   team_h.*
                        from team_h
                        where res in ('l')
                     """).df()
team_hl = con.execute("""select teamID,teamNAME,count(*) as los,sum(dg) as dg
                        from team_hl
                        group by teamID,teamNAME
                     """).df()
#####
team_a = con.execute("""select   e1a.*, case when e1.homeGoals < e1.awayGoals  then 'w'
                                             when e1.homeGoals > e1.awayGoals then 'l'
                                             else 'd'
                                        end as res,
                                        e1.awayGoals - e1.homeGoals  as dg
                        from e1a  left join e1 on (e1a.teamID = e1.hometeamID)
                     """).df()

team_aw = con.execute("""select   team_a.*
                        from team_a
                        where res in ('w')
                     """).df()
team_aw = con.execute("""select teamID,teamNAME,count(*) as vic, sum(dg) as dg
                        from team_aw
                        group by teamID,teamNAME
                     """).df()

team_ad = con.execute("""select   team_a.*
                        from team_a
                        where res in ('d')
                     """).df()
team_ad = con.execute("""select teamID,teamNAME,count(*) as dra, sum(dg) as dg
                        from team_ad
                        group by teamID,teamNAME
                     """).df()

team_al = con.execute("""select   team_a.*
                        from team_a
                        where res in ('d')
                     """).df()
team_al = con.execute("""select teamID,teamNAME,count(*) as los, sum(dg) as dg
                        from team_al
                        group by teamID,teamNAME
                     """).df()
    
e1a["win"] = team_hw["vic"] + team_aw["vic"]
e1a["draw"] = team_hd["dra"] + team_ad["dra"]
e1a["puntos"] = 3* e1a["win"] + e1a["draw"]
e1a["dif_gol"] = team_hw["dg"] + team_hl["dg"] + team_aw["dg"] + team_al["dg"] 

e1a = con.execute("""select   e1a.*
                     from e1a
                     order by puntos desc
                     """).df()
e1a

,teamID,teamNAME,win,draw,puntos,dif_gol
0,179,Lorient,21,14,77,36.00
1,164,Lille,22,8,74,16.00
2,166,Montpellier,18,16,70,25.00
3,178,Lyon,20,6,66,26.00
4,161,Paris Saint Germain,17,14,65,12.00
5,170,Nice,16,12,60,13.00
6,171,Monaco,16,10,58,9.00
7,160,Marseille,16,6,54,10.00
8,225,Strasbourg,12,16,52,15.00
9,177,Reims,15,6,51,10.00


In [10]:
#Punto 6
e2 = con.execute("""select   appear.*, games.leagueID,games.season
                     from appear left join games on (appear.gameID = games.gameID )
                     where games.season == 2016 and games.leagueID == 5
                     """).df()
e2a = con.execute("""select playerID,playerNAME, sum(goals) as goals, sum(time) as time
                        from e2
                        group by playerID,playerNAME
                     """).df()

e2a ["prom"] = 90 * (e2a['goals']/e2a['time'])

e2a = con.execute("""select e2a.*
                        from e2a
                        where goals > 10
                        order by prom desc
                     """).df()


e2a

,playerID,playerNAME,goals,time,prom
0,3294,Edinson Cavani,35.00,"2,983.00",1.06
1,3277,Alexandre Lacazette,28.00,"2,416.00",1.04
2,696,Falcao,21.00,"1,969.00",0.96
3,3423,Kylian Mbappe-Lottin,15.00,"1,503.00",0.90
4,1817,Mario Balotelli,15.00,"1,748.00",0.77
5,717,Bafétimbi Gomis,20.00,"2,613.00",0.69
6,3244,Nicolas de Preville,14.00,"2,077.00",0.61
7,3570,Benjamin Moukandjo,13.00,"2,017.00",0.58
8,3310,Alassane Pléa,11.00,"1,853.00",0.53
9,5701,Ivan Santini,15.00,"2,905.00",0.46


In [11]:
#Punto 7
e2 = con.execute("""select   appear.*, games.leagueID,games.season
                     from appear left join games on (appear.gameID = games.gameID )
                     where games.season == 2017 and games.leagueID == 4
                     """).df()
e2a = con.execute("""select playerID,playerNAME, sum(goals) as goals, sum(time) as time
                        from e2
                        group by playerID,playerNAME
                     """).df()

e2a ["prom"] = 90 * (e2a['goals']/e2a['time'])

e2a = con.execute("""select e2a.*
                        from e2a
                        where goals > 10
                        order by prom desc
                     """).df()

e2a

,playerID,playerNAME,goals,time,prom
0,2097,Lionel Messi,34.00,"2,995.00",1.02
1,2371,Cristiano Ronaldo,26.00,"2,304.00",1.02
2,2251,Gareth Bale,16.00,"1,803.00",0.80
3,2098,Luis Suárez,25.00,"2,902.00",0.78
4,2384,Santi Mina,12.00,"1,548.00",0.70
5,1709,Cristhian Stuani,21.00,"2,728.00",0.69
6,2270,Antoine Griezmann,19.00,"2,532.00",0.68
7,2290,Iago Aspas,22.00,"2,939.00",0.67
8,1642,Simone Zaza,13.00,"2,082.00",0.56
9,1125,Carlos Bacca,15.00,"2,513.00",0.54


In [12]:
#Punto 8
e2 = con.execute("""select   appear.*, games.leagueID,games.season
                     from appear left join games on (appear.gameID = games.gameID )
                     where games.season == 2018 and games.leagueID == 3
                     """).df()
e2a = con.execute("""select playerID,playerNAME, sum(goals) as goals, sum(time) as time
                        from e2
                        group by playerID,playerNAME
                     """).df()

e2a ["prom"] = 90 * (e2a['goals']/e2a['time'])

e2a = con.execute("""select e2a.*
                        from e2a
                        where goals > 10
                        order by prom desc
                     """).df()

e2a

,playerID,playerNAME,goals,time,prom
0,2380,Paco Alcácer,18.00,"1,181.00",1.37
1,3834,Ishak Belfodil,16.00,"1,867.00",0.77
2,6228,Luka Jovic,17.00,"2,256.00",0.68
3,227,Robert Lewandowski,22.00,"2,959.00",0.67
4,319,Marco Reus,17.00,"2,335.00",0.66
5,956,Andrej Kramaric,17.00,"2,418.00",0.63
6,6144,Sébastien Haller,15.00,"2,176.00",0.62
7,65,Timo Werner,16.00,"2,537.00",0.57
8,5250,Yussuf Poulsen,15.00,"2,450.00",0.55
9,5220,Kai Havertz,17.00,"2,906.00",0.53


In [13]:
#Punto 9
e2 = con.execute("""select   appear.*, games.leagueID,games.season
                     from appear left join games on (appear.gameID = games.gameID )
                     where games.season == 2019 and games.leagueID == 2
                     """).df()
e2a = con.execute("""select playerID,playerNAME, sum(goals) as goals, sum(time) as time
                        from e2
                        group by playerID,playerNAME
                     """).df()

e2a ["prom"] = 90 * (e2a['goals']/e2a['time'])

e2a = con.execute("""select e2a.*
                        from e2a
                        where goals > 10
                        order by goals desc
                     """).df()

e2a

,playerID,playerNAME,goals,time,prom
0,1209,Ciro Immobile,36.00,"3,179.00",1.02
1,2371,Cristiano Ronaldo,31.00,"2,920.00",0.96
2,594,Romelu Lukaku,23.00,"2,985.00",0.69
3,6980,Francesco Caputo,21.00,"2,940.00",0.64
4,1148,Duván Zapata,18.00,"2,047.00",0.79
5,1229,Luis Muriel,18.00,"1,225.00",1.32
6,1592,João Pedro,18.00,"3,170.00",0.51
7,1253,Edin Dzeko,16.00,"2,857.00",0.50
8,1186,Andrea Belotti,15.00,"3,010.00",0.45
9,1426,Josip Ilicic,15.00,"1,682.00",0.80


In [14]:
#Punto 10
e2 = con.execute("""select   appear.*, games.leagueID,games.season
                     from appear left join games on (appear.gameID = games.gameID )
                     where games.season == 2020 and games.leagueID == 1
                     """).df()
e2a = con.execute("""select playerID,playerNAME, sum(goals) as goals, sum(time) as time
                        from e2
                        group by playerID,playerNAME
                     """).df()

e2a ["prom"] = 90 * (e2a['goals']/e2a['time'])

e2a = con.execute("""select e2a.*
                        from e2a
                        where goals > 10
                        order by prom desc
                     """).df()

e2a

,playerID,playerNAME,goals,time,prom
0,2251,Gareth Bale,11.00,909.00,1.09
1,620,Kelechi Iheanacho,12.00,"1,452.00",0.74
2,647,Harry Kane,23.00,"3,097.00",0.67
3,1250,Mohamed Salah,22.00,"3,085.00",0.64
4,3277,Alexandre Lacazette,13.00,"1,939.00",0.60
5,314,Ilkay Gündogan,13.00,"2,033.00",0.58
6,1228,Bruno Fernandes,18.00,"3,117.00",0.52
7,468,Callum Wilson,12.00,"2,079.00",0.52
8,5555,Dominic Calvert-Lewin,16.00,"2,878.00",0.50
9,822,Patrick Bamford,17.00,"3,085.00",0.50


In [15]:
games.head()

,gameID,leagueID,season,date,homeTeamID,awayTeamID,homeGoals,awayGoals,homeProbability,drawProbability,...,WHH,WHD,WHA,VCH,VCD,VCA,PSCH,PSCD,PSCA,leagueNAME
0,81,1,2015,2015-08-08 15:45:00,89,82,1,0,0.28,0.40,...,1.62,3.60,6.00,1.67,4.00,5.75,1.64,4.07,6.04,Premier League
1,82,1,2015,2015-08-08 18:00:00,73,71,0,1,0.36,0.35,...,1.91,3.50,4.00,2.00,3.50,4.20,1.82,3.88,4.70,Premier League
2,83,1,2015,2015-08-08 18:00:00,72,90,2,2,0.30,0.43,...,1.73,3.50,5.00,1.73,3.90,5.40,1.75,3.76,5.44,Premier League
3,84,1,2015,2015-08-08 18:00:00,75,77,4,2,0.64,0.21,...,2.00,3.10,2.70,2.00,3.40,4.33,1.79,3.74,5.10,Premier League
4,85,1,2015,2015-08-08 18:00:00,79,78,1,3,0.15,0.22,...,2.60,3.10,2.88,2.60,3.25,3.00,2.46,3.39,3.14,Premier League


In [16]:
appear.head()

,gameID,playerID,goals,ownGoals,shots,xGoals,xGoalsChain,xGoalsBuildup,assists,keyPasses,...,position,positionOrder,yellowCard,redCard,time,substituteIn,substituteOut,leagueID,leagueNAME,playerNAME
0,81,557,0,0,0,0.00,0.11,0.11,0,1,...,"""""DR""""",2,0,0,82,"""""222605""""","""""0""""",557,NaN,Matteo Darmian
1,81,548,0,0,0,0.00,0.13,0.13,0,0,...,"""""DC""""",3,0,0,90,"""""0""""","""""0""""",548,NaN,Daley Blind
2,81,628,0,0,0,0.00,0.11,0.11,0,0,...,"""""DC""""",3,0,0,90,"""""0""""","""""0""""",628,NaN,Chris Smalling
3,81,551,0,0,0,0.00,0.16,0.16,0,0,...,"""""DMC""""",7,0,0,90,"""""0""""","""""0""""",551,NaN,Morgan Schneiderlin
4,81,654,0,0,0,0.00,0.04,0.00,0,1,...,"""""DMC""""",7,0,0,61,"""""222606""""","""""0""""",654,NaN,Michael Carrick


In [17]:
uefa_l.head()

,leagueID,leagueNAME
0,1,Premier League
1,2,Serie A
2,3,Bundesliga
3,4,La Liga
4,5,Ligue 1


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cd2fca3f-dc64-4c71-8bd4-adda531fa911' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>